In [1]:
import os
import pandas as pd
import ARTISTIC

if __name__ == "__main__":
    db     = ARTISTIC.AptamerDatabase("UTexas_Datasheet.xlsx")
    target = input("Enter target ligand: ").strip()
    hits   = db.search_by_target(target)

    # create a directory for this ligand
    out_dir = target.replace(" ", "_")
    os.makedirs(out_dir, exist_ok=True)

    # titration parameters
    multipliers     = [0, 1, 2, 2.5, 5, 10, 25, 50, 100]
    monovalent_conc = [0, 5, 10, 25, 50, 100]
    divalent_conc   = [0, 1, 2, 3, 4, 5]

    if hits.empty:
        print(f"No aptamers found for '{target}'.")
    else:
        for _, row in hits.iterrows():
            name = row.get('name_of_aptamer') or row.get('aptamer_name', '<unnamed>')
            kd   = row.get('kd_nm', None)
            seq  = row.get('aptamer_sequence', '')
            buf  = (
                row.get('binding_buffer_conditions')
                or row.get('binding_buffer')
                or row.get('binding_conditions')
                or ''
            )
            salt, salt_conc = ARTISTIC.highest_salt(buf)

            if pd.notna(kd) and salt:
                designed, dna_tmpl, ins, ins_comp = ARTISTIC.test_insulations(seq)

                if designed:
                    # pick conc list & filename
                    if salt in ("NACL", "KCL"):
                        conc_list, idx_name = monovalent_conc, f"[{salt}] (mM) vs [Ligand] (nM)"
                        fname = f"{name}_{salt}_titration.xlsx"
                    else:
                        conc_list, idx_name = divalent_conc, "[MgCl2] (mM) vs [Ligand] (nM)"
                        fname = f"{name}_MgCl2_titration.xlsx"

                    # full path inside ligand folder
                    out_path = os.path.join(out_dir, fname)

                    # build titration table
                    table = pd.DataFrame(
                        index=conc_list,
                        columns=[kd * m for m in multipliers]
                    )
                    table.index.name = idx_name

                    # metadata block
                    meta = pd.DataFrame({
                        "Variables": [
                            "Aptamer", "Kd (nM)", "Sequence",
                            "Buffer Conditions", "dART template strand",
                            "Promoter non-template strand", "Output non-template strand"
                        ],
                        " ": [
                            name, kd, seq,
                            buf, dna_tmpl,
                            f"{ARTISTIC.Prom_nt}{ins}", f"{ins_comp}{ARTISTIC.Output_nt}"
                        ]
                    })

                    # write to Excel in the target's folder
                    with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
                        meta.to_excel(writer, index=False, sheet_name="Titration", startrow=0)
                        table.to_excel(writer, sheet_name="Titration", startrow=len(meta) + 2)

                    print(f"Protocol saved under: {out_path}\n")
                else:
                    print("")

Enter target ligand:  MUC1


Protocol saved under: MUC1/aptamer 1_NACL_titration.xlsx

Protocol saved under: MUC1/aptamer 2_NACL_titration.xlsx

Protocol saved under: MUC1/MUC1-5TR-1_NACL_titration.xlsx

Protocol saved under: MUC1/MUC1-5TR-2_NACL_titration.xlsx

Protocol saved under: MUC1/MUC1-5TR-3_NACL_titration.xlsx

Protocol saved under: MUC1/MUC1-5TR-4_NACL_titration.xlsx

